In [1]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
        
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import glob
import argparse
import os

from shutil import copyfile, move
from pathlib import Path
from mpl_finance import candlestick2_ochl, volume_overlay

C:\Users\SeungHyuck\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [13]:
def removeOutput(finput):
    if(Path(finput)).is_file():
        os.remove(finput)
    
def csv_initiator(market, ticker, head_date, tail_date):
    df = fdr.DataReader(ticker, head_date, tail_date, exchange=market)\
    
    outputname = "{}_{}.csv".format(market, ticker)
    filepath = os.getcwd() + '\\dataset\\raw_data\\'
    
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    removeOutput(filepath + outputname)
    df.to_csv(filepath + outputname)
    
    print("csv file saved as : {}".format(filepath + outputname))
    
    return filepath+outputname

In [14]:
def seqEnd_trend(csv_path, seq_len):
    print("Creating label . . .")
    print("type : sequence_end")
    
    df = pd.read_csv(csv_path, parse_dates=True, index_col=0)
    df.fillna(0)
    df.reset_index(inplace=True)
    
    filename = csv_path.split('\\')[-1][0:-4]
    
    outputname = "{}_label_seq{}.txt".format(filename, seq_len) # ticker 명과 sequence 길이를 파일이름에 포함
        
    filepath = os.getcwd() + '\\dataset\\labeled_data\\'
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    removeOutput(filepath + outputname)
        
    df['Date'] = df['Date'].map(mdates.date2num) # Y-M-D 포멧에서 num 포멧으로 변경
    
    for i in range(0, len(df)-int(seq_len)):
        tmp_df = df.iloc[i:i + int(seq_len)]  # seq_len 만큼 데이터프레임 슬라이싱
        starting = 0
        endvalue = 0
        label = ""
        starting = tmp_df["Open"].iloc[-2] # seq 마지막날의 전날 종가
        endvalue = tmp_df["Close"].iloc[-1] # seq 마지막날 종가
        tmp_rtn = endvalue / starting - 1 
          
        if tmp_rtn > 0:
            label = 1
        else:
            label = 0
        
        with open(filepath+outputname, 'a') as the_file:
            the_file.write("{}--{},{}".format(outputname[0:-4], i, label))
            the_file.write("\n")

    print("Create label finished.")
    return outputname

In [4]:
def ohlc2cs(fname, seq_len, dimension, use_volume):
    print("Converting ohlc to candlestick")
    
    symbol = fname.split('.')[0]

    filepath = os.getcwd() + '\\dataset\\'
    
    if not os.path.exists("{}/dataset/{}_{}/{}/".format(path, seq_len, dimension, symbol)):
        os.makedirs("{}/dataset/{}_{}/{}/".format(path, seq_len, dimension, symbol))
        
    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)
    
    plt.style.use('dark_background') # 배경을 검은색으로
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num) # Y-M-D 포멧에서 num 포멧으로 변경
    
    for i in range(0, len(df)-int(seq_len)):
        tmp_df = df.iloc[i:i + int(seq_len)]
        if len(tmp_df) == int(seq_len):
            my_dpi = 109 # 자신의 모니터에 맞는 dpi 설정 27inch qhd -> 109
            fig = plt.figure(figsize=(dimension / my_dpi, dimension / my_dpi), dpi=my_dpi) # figure size 설정
            ax1 = fig.add_subplot(1, 1, 1)
            candlestick2_ochl(ax1, tmp_df['Open'], tmp_df['Close'], tmp_df['High'], tmp_df['Low'],
                                width=1, colorup='#77d879', colordown='#db3f3f')
            
            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')
            
            
            if use_volume:
                ax2 = ax1.twinx()
                bc = volume_overlay(ax2, tmp_df['Open'], tmp_df['Close'], tmp_df['Volume'],
                                        colorup='#77d879', colordown='#db3f3f',
                                        alpha=0.5,  width=1)
                ax2.add_collection(bc)
                ax2.grid(False)
                ax2.set_xticklabels([])
                ax2.set_yticklabels([])
                ax2.xaxis.set_visible(False)
                ax2.yaxis.set_visible(False)
                ax2.axis('off')
                
            pngfile = 'dataset/{}_{}/{}/{}/{}-{}.png'.format(
                        seq_len, dimension, symbol, dataset_type, fname[:-4], i)
            fig.savefig(pngfile, pad_inches=0, transparent=False)
            plt.close(fig)
    print("Converting ohlc to candlestick finished")                
                
                

In [16]:
# main
market = 'KRX'
ticker = '005930'

seq_len = '30'
head_date = '2020-01-01'
tail_date = '2022-03-01'
dimension = 536
use_volume = False

# fdr -> data.csv ,data.csv -> labeled_data.txt, data.csv -> candlechart_img.png

data_csv_path = csv_initiator(market, ticker, head_date, tail_date) # 이 함수를 추가해서 FDR 이용하여 데이터를 불러오고 csv 파일로 저장하는 부분을 추가
label_set = seqEnd_trend(data_csv_path, seq_len)
# ohlc2cs(data_csv, seq_len, dimension, use_volume)

csv file saved as : C:\Users\SeungHyuck\Documents\github_remote\AI-Trade\DL Quant Strategy\dataset\raw_data\KRX_005930.csv
Creating label . . .
type : sequence_end
Create label finished.
